# Data exploration using widgets in Matplotlib-II

In my [previous post](../widgets-matplotlib.html), I had discussed data exploration using Matplotlib widgets. In this post, I will take that example as a base and build more advanced widget controls.

### Setting the backend to TK. 

For some reasons, it works better than the default OSX one.

In [1]:
%matplotlib tk

### Customary imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

### Creating the data

Imagine we have energy data from 15 homes, each having data for fridge and microwave, for 365 days, one reading per day.

In [3]:
# Fixing the seed for reproducibility
np.random.seed(0)

with pd.HDFStore('temp-store.h5', mode='w') as st:

    # 15 home-> 2 columns, 365 rows (daily one reading)
    for home in range(15):
        df = pd.DataFrame(np.random.randn(365, 2), columns=['fridge','microwave'],
                          index=pd.DatetimeIndex(start='2017',freq='D', periods=365))
        df = df.abs()
        st['/home_%d' %home] = df

In [4]:
st = pd.HDFStore('temp-store.h5', mode='r')

In [5]:
fig, ax  = plt.subplots()
st['/home_2'].plot(ax=ax)

Notice, that since I used `%matplotlib TK` backend, I don't  see the plot embedded in the notebook. Thus I'll save the current figure as an image and then link it here.

In [6]:
plt.savefig("post2-all.png")

![](post2-all.png)

This sure does not look pretty. 

### Proposed solution

![](mpl-widget.gif)

Great. It seems to do the intended job. Let us now look at the individual pieces and how we can tie them up.

### Creating the initial frame

In the first frame we would like to plot the data for the first sample.
```python
fig, ax = plt.subplots()
df[0].plot(ax=ax, title="Sample number: 0")
```

### Creating the buttons at the bottom

First, we'd want to make space for the button at the bottom and place them there. We can do this as follows:

```python
from matplotlib.widgets import Button

fig.subplots_adjust(bottom=0.2)

axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])

bnext = Button(axnext, '>')
bprev = Button(axprev, '<')
```

### Linking the buttons to functions

We'd next want to call some function each time either of the two buttons are pressed. We would also need a notion of currently selected data point. The idea would be that each time, `>` is pressed, we advance the currently selected point and plot correspondingly.


We'd have to define `next() and prev()` where we increment and decrement the selected data point.

```python

class Index:
    data = df
    selected = 0
    
    def next(self, event):
        self.selected += 1
        ax.cla()
        df[self.selected].plot(ax=ax)
        ax.set_title("Sample number: %d" %self.selected)

    def prev(self, event):
        self.selected -= 1
        ax.cla()
        df[self.selected].plot(ax=ax)
        ax.set_title("Sample number: %d" %self.selected)
```

Here, `ax.cla()` clears the data for the current data point before drawing for the next one. `df[self.selected].plot(ax=ax)` plots for the newly selected data. `ax.set_title("Sample number: %d" %self.selected)` would change the title to reflect the currently used data point.

We can link to callback as follows: 

```python
callback = Index()

bnext.on_clicked(callback.next)
bprev.on_clicked(callback.prev)
```


### Ensuring we do not select home out of range

If you notice, we simply incremented and decremented the selected data point without considering going beyond (0, number of data points). So, we need to change the call back functions to check that we do not go beyond the range. This would require the following changes to `next()` with the changes to `prev()` being similar.

```python
data_min = 0
data_max = data.shape[1]-1
selected = 0
def next(self, event):
    if self.selected >=self.data_max:
        self.selected = self.data_max
        ax.set_title('Last sample reached. Cannot go forwards')
    else:
        self.selected += 1
        ax.cla()
        df[self.selected].plot(ax=ax)
        ax.set_title("Sample number: %d" %self.selected)
 ```

There you go. This was fairly simple and fun to do, and yet can be very helpful! 

### Complete code

In [15]:
from matplotlib.widgets import Button, CheckButtons

fig, ax = plt.subplots()
fig.subplots_adjust(bottom=0.2)
fig.subplots_adjust(left=0.2)

home_0 = st['/home_0']

rax = plt.axes([0.02, 0.4, 0.13, 0.2], aspect='equal')

labels = tuple(home_0.columns)
states = tuple([True]*len(labels))
check = CheckButtons(rax, labels, states)


st['/home_0'].plot(ax=ax, title="Sample number: 0").legend(loc=2)
lines = ax.get_lines()

class Index:
    store = st
    data_min = 0
    data_max = len(store.keys())-1
    selected = 0
    st, la = states, labels
    states_dict = dict(zip(la, st))
    def selected_column(self, label):
        self.states_dict[label] = not self.states_dict[label]
        self.plot()
    
    def plot(self):
        ax.cla()
        st['/home_%d' %self.selected].plot(ax=ax, title="Sample number: %d" %self.selected).legend(loc=2)
        lines = ax.get_lines()
        for i ,(l, s) in enumerate(self.states_dict.items()):
            lines[i].set_visible(s)
        plt.legend(loc=1)
        
        
    def next(self, event):
        if self.selected >=self.data_max:
            self.selected = self.data_max
            ax.set_title('Last sample reached. Cannot go forwards')
        else:
            self.selected += 1
            self.plot()
            

    def prev(self, event):
        if self.selected <=self.data_min:
            self.selected = 0
            ax.set_title('First sample reached. Cannot go backwards')
        else:
            self.selected -= 1
            self.plot()
        

callback = Index()
axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.81, 0.05, 0.1, 0.075])

bnext = Button(axnext, '>')
bnext.on_clicked(callback.next)

bprev = Button(axprev, '<')
bprev.on_clicked(callback.prev)

check.on_clicked(callback.selected_column)

0

In [35]:
check.ax


In [26]:
l = ax.get_lines()[0]

In [ ]:
l.

In [10]:
from matplotlib.widgets import CheckButtons

fig, ax = plt.subplots()
fig.subplots_adjust(left=0.2)

# plot some hidden curves
freqs = np.arange(1, 4)
t = np.linspace(0, 2, 1000)
s = np.sin(2 * np.pi * freqs[:, None] * t)
lines = plt.plot(t, s.T, lw=2, visible=False)
ax.set_ylim(-1.5, 1.5)

# Build check button axes
rax = plt.axes([0.02, 0.4, 0.13, 0.2], aspect='equal')
labels = ('2 Hz', '4 Hz', '6 Hz')
check = CheckButtons(rax, labels, (False, False, False))

def func(label):
    i = labels.index(label)
    lines[i].set_visible(not lines[i].get_visible())
    fig.canvas.draw()

check.on_clicked(func)

0

In [ ]:
def __init__(self):
        colourax = plt.axes([0.5, 0.4, 0.09, 0.2])
        measurementax = plt.axes([0.5, 0.6, 0.09, 0.2])
        clabels, cvals = ('Red', 'Green', 'Blue'), (False,)*3
        mlabels, mvals = ('1', '2', '3'), (False,)*3
        self.cstates = dict(zip(clabels,cvals))
        self.mstates = dict(zip(mlabels,mvals))
        self.colours = CheckButtons(colourax, clabels, cvals)
        self.colours.on_clicked(self.onColor)
        self.measurements = CheckButtons(measurementax, mlabels, mvals)
        self.measurements.on_clicked(self.onMeasurement)
